In [1]:
# data analysis imports
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# NLP Imports
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import re

from sklearn.feature_extraction.text import CountVectorizer

from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from PIL import Image
import wordninja

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ayaanhaque/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
df = pd.read_csv('../data/bad.csv')

In [21]:
df.head()

,title,selftext,author,num_comments,is_suicide,url,selftext_clean,title_clean,author_clean
0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,SQLwitch,133,0,https://www.reddit.com/r/depression/comments/d...,understand people reply immediately op invitat...,broken least understood rule helper may invite...,sql witch
1,Regular Check-In Post,Welcome to /r/depression's check-in post - a p...,circinia,1644,0,https://www.reddit.com/r/depression/comments/e...,welcome r depression check post place take mom...,regular check post,c irc
2,I hate it so much when you try and express you...,I've been feeling really depressed and lonely ...,TheNewKiller69,8,0,https://www.reddit.com/r/depression/comments/f...,feeling really depressed lonely lately job ful...,hate much try express feeling parent turn arou...,new killer 69
3,I went to the hospital because I was having re...,I literally broke down crying and asked to go ...,Jazzlecrab,15,0,https://www.reddit.com/r/depression/comments/f...,literally broke cry asked go home another nurs...,went hospital wa really bad panic attack theyc...,jazz le crab
4,Cake day for me,Any kind soul want to give a depressed person ...,Depressed_Kid786,37,0,https://www.reddit.com/r/depression/comments/f...,kind soul want give depressed person karma edi...,cake day,depressed kid 786


In [15]:
for row in df.index:
    if df["is_suicide"][row] == 1:
        df["is_suicide"][row] = 0
    else:
        df["is_suicide"][row] = 1

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [16]:
df.head()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,poll_data,author_cakeday,is_suicide
0,NaN,CasualConversation,Let's chat!,t2_5r8xp,False,NaN,0,False,r/CasualConversation Lounge,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",...,True,https://www.reddit.com/r/CasualConversation/co...,1346321,1.597424e+09,0,NaN,False,NaN,NaN,0
1,NaN,CasualConversation,Monthly Meta is back - [follow the collection]...,t2_5r8xp,False,NaN,0,False,September Monthly Meta - r/CasualConversation ...,"[{'a': ':shield:', 'e': 'emoji', 'u': 'https:/...",...,True,https://www.reddit.com/r/CasualConversation/co...,1346321,1.598980e+09,0,NaN,False,NaN,NaN,0
2,NaN,CasualConversation,"Look, I love my family and they're awesome. Bu...",t2_7s6xkdc5,False,NaN,0,False,I got a hotel room here in my town because I n...,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346321,1.599439e+09,0,NaN,False,NaN,NaN,0
3,NaN,CasualConversation,Lately I’ve noticed many guys around my circle...,t2_5kmlnpy4,False,NaN,1,False,Your height is totally fine,"[{'a': ':chat:', 'e': 'emoji', 'u': 'https://e...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346321,1.599397e+09,2,NaN,False,NaN,NaN,0
4,NaN,CasualConversation,I was at a Halloween party with my best friend...,t2_5yz95sxz,False,NaN,0,False,I remember this conversation with an old acqua...,[],...,False,https://www.reddit.com/r/CasualConversation/co...,1346321,1.599398e+09,3,NaN,False,NaN,NaN,0


In [17]:
df.tail()

,approved_at_utc,subreddit,selftext,author_fullname,saved,mod_reason_title,gilded,clicked,title,link_flair_richtext,...,stickied,url,subreddit_subscribers,created_utc,num_crossposts,media,is_video,poll_data,author_cakeday,is_suicide
831,NaN,CasualConversation,As a male it’s insanely frustrating to browse ...,t2_dwlc2,False,NaN,0,False,"As a male, I’m so tired of the lack of unique ...",[],...,False,https://www.reddit.com/r/CasualConversation/co...,1346322,1.597228e+09,0,NaN,False,NaN,NaN,0
832,NaN,CasualConversation,I have 4 boys ranging from ages 3 to 14. Tonig...,t2_js8ar,False,NaN,0,False,My teenager made me so proud tonight with a si...,"[{'a': ':all:', 'e': 'emoji', 'u': 'https://em...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346322,1.597126e+09,0,NaN,False,NaN,NaN,0
833,NaN,CasualConversation,My family has owned a fine dining italian rest...,t2_28kjonye,False,NaN,8,False,"After 30 years of being open, my family’s rest...","[{'a': ':all:', 'e': 'emoji', 'u': 'https://em...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346322,1.596916e+09,3,NaN,False,NaN,NaN,0
834,NaN,CasualConversation,Just live streams of people at home getting in...,t2_mnv1s,False,NaN,2,False,This coronavirus things has made me realize pe...,"[{'a': ':all:', 'e': 'emoji', 'u': 'https://em...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346322,1.584640e+09,15,NaN,False,NaN,NaN,0
835,NaN,CasualConversation,At first I was mad that I would still have to ...,t2_bx21xut,False,NaN,16,False,I’m a lunch lady at an elementary school schoo...,"[{'a': ':all:', 'e': 'emoji', 'u': 'https://em...",...,False,https://www.reddit.com/r/CasualConversation/co...,1346322,1.584124e+09,6,NaN,False,NaN,NaN,0


In [18]:
df["subreddit"]

0      CasualConversation
1      CasualConversation
2      CasualConversation
3      CasualConversation
4      CasualConversation
              ...        
831    CasualConversation
832    CasualConversation
833    CasualConversation
834    CasualConversation
835    CasualConversation
Name: subreddit, Length: 836, dtype: object

In [19]:
df["is_suicide"].sum()

0